In [0]:
import os, uuid
import time, datetime
import multiprocessing
from multiprocessing import cpu_count
import pandas as pd
from azure.cosmos import CosmosClient

In [0]:
container_name="espdata"
storage_account_name = "gw2data2lake2storage"
storage_account_access_key = "8iT17dy9BxdNbEpT68hjdV72+doHN7cGICXAvMF5iI67Jw0gzVYw5jvXE3IYZTD0khpNXKj6lb7vUS9xBzPhpQ=="

connect_str = "DefaultEndpointsProtocol=https;AccountName=gw2data2lake2storage;AccountKey=8iT17dy9BxdNbEpT68hjdV72+doHN7cGICXAvMF5iI67Jw0gzVYw5jvXE3IYZTD0khpNXKj6lb7vUS9xBzPhpQ==;EndpointSuffix=core.windows.net"

file_location = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key)

In [0]:
time = str(datetime.datetime.now())
print(time)
new_time = time[:10].split('-')
year , month , day = int(new_time[0]) , int(new_time[1]) , int(new_time[2])
yesterday = str(year) + '/' + str(month) + '/' +str(day)
print(yesterday)

2021-04-06 09:36:41.153527
2021/4/6

In [0]:
spark.catalog.setCurrentDatabase("camera")

In [0]:
cameras = dbutils.fs.ls(file_location)
data = [] # camera_name , year , month , day , count 
for camera in cameras:
  camera_name = camera[1]
  img_location = file_location + camera_name  + yesterday# wasbs://espdata@gw2data2lake2storage.blob.core.windows.net/00000000000000T/
  camera_name = camera_name.replace('/','')
  try:
    img = dbutils.fs.ls(img_location)
    row = []
    row.extend([camera_name,year,month,day,len(img)])
    data.append(row)
  except:
    row = []
    row.extend([camera_name,year,month,day,0])
    data.append(row)
data_final = pd.DataFrame(data,columns = ['camera_name' , 'year' , 'month' , 'day' , 'up_load_image_num'])


# convert into spark df
cc=data_final.values.tolist()
dd=list(data_final.columns)
spark_df  = spark.createDataFrame(cc, dd)
spark_df

Out[124]: DataFrame[camera_name: string, year: bigint, month: bigint, day: bigint, up_load_image_num: bigint]

In [0]:
spark_df.createOrReplaceTempView("cameraStatus_yesterday")

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `cameraStatus_yes`

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: cameraStatus_yes; line 3 pos 14;
'Project [*]
+- 'UnresolvedRelation [cameraStatus_yes]

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:113)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1$adapted(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:190)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:189)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:181)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:204)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:201)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:132)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:92)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:89)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:81)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:676)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:671)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:144)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:451)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:49)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:277)
	at com.databricks.logging.UsageLo

In [0]:
%sql
show tables

database,tableName,isTemporary
camera,camerastatus,false
camera,gwimage,false
,camerastatus_yesterday,true


In [0]:
%sql
drop table if exists cameraStatus_yes

In [0]:
%sql
REFRESH TABLE cameraStatus_yes

In [0]:
spark_df.write.saveAsTable("cameraStatus_yes")

In [0]:
spark_df.write.format("parquet").mode('append').save("cameraStatus_yesterday")

In [0]:
# df = spark.sql('select * from cameraStatus')
# pandas_df = df.toPandas()  
# pandas_df

In [0]:
# data_final

In [0]:
# pandas_df = pd.concat([pandas_df,data_final],axis=0)
# pandas_df

In [0]:
# cc=pandas_df.values.tolist()
# dd=list(pandas_df.columns)
# spark_df  = spark.createDataFrame(cc, dd)
# spark_df

In [0]:
# %sql
# drop table if exists cameraStatus

In [0]:
# spark_df.write.format("parquet").saveAsTable("cameraStatus")